# Pydantic

Pydantic to biblioteka w Pythonie, która jest używana do konwersji, walidacji i serializacji danych. Podstawowym sposobem jej użycia jest zdefiniowanie modeli czyli klas dziedziczących po `BaseModel`, których struktura definiuje oczekiwany schemat danych:

In [9]:
from pydantic import BaseModel, model_validator
from datetime import datetime
from typing import Self

class OHLC(BaseModel):
    date: datetime
    open: float
    high: float
    low: float
    close: float
    
    @model_validator(mode="after")
    def validate_start_before_end(self: Self) -> Self:
        if self.close < self.low or self.open < self.low:
            raise ValueError("low must be the lowest value")
        if self.close > self.high or self.open > self.high:
            raise ValueError("high must be the highest value")
        return self

# dane odebrane np.: w postaci zapytania HTTP
raw_data = {
    "date": "2023-10-03 20:00:00",
    "open": 121.0,
    "high": 105.0,
    "low": 99.0,
    "close": 103.5
}

# Konwersja na obiekt klasy OHLC powoduje walidację i ew. rzucenie wyjątkiem ValueError
try:
    ohlc_entry = OHLC(**raw_data)
    print("Data is correct!")
except ValueError as e:
    print(f"Validation error: {e}")


Validation error: 1 validation error for OHLC
  Value error, high must be the highest value [type=value_error, input_value={'date': '2023-10-03 20:0...': 99.0, 'close': 103.5}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


W powyższym przykładzie klasa OHLC jest modelem i definiuje 2 metody walidujące określone aspekty danych. Następnie pod koniec przykładu konwertujemy słownik `raw_data` na obiekt OHLC, uruchamiając logikę walidującą. Pydantic bazuje na metodach oznaczonych jako `@model_validator` lub `@field_validator` oraz typach danych pól zdefiniowanych w modelu.

## Konwersja typów
Jeśli to możliwe, przekazane dane zostają automatycznie skonwertowane na typy zadeklarowane przy użyciu anotacji typowych:

In [10]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

p = Person(name="Alan", age="25")
print(f"age is of type {type(p.age)} and has value {p.age}")

age is of type <class 'int'> and has value 25


In [11]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

try:
    p = Person(name="Alan", age="dwadzieścia pięć")
    print(f"age is of type {type(p.age)} and has value {p.age}")
except ValueError as e:
    print(e.errors())

[{'type': 'int_parsing', 'loc': ('age',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'dwadzieścia pięć', 'url': 'https://errors.pydantic.dev/2.4/v/int_parsing'}]


Automatycznie dostajemy możliwość konwersji obiektów danego modelu na słowniki:

In [12]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

    
p = Person(name="Alan", age=25)
p.model_dump()

{'name': 'Alan', 'age': 25}

... lub prosto do JSON-owego stringa:

In [13]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

    
p = Person(name="Alan", age=25)
p.model_dump_json()

'{"name":"Alan","age":25}'

Możliwe jest także tworzenie instancji modelu bezpośrednio z JSON-a:

In [ ]:
Person.model_validate_json('{"name":"Alan","age":25}')

... lub słownika:

In [ ]:
Person.model_validate({'name': 'Adam', 'age': 23})

## Strict mode
W trybie *strict* nie następuje standardowa konwersja typów - dla obiektów Pythona wymagana jest pełna zgodność typowa. Wyjątkiem jest konwersja z JSON-a, gdzie konieczne jest dopuszczanie pewnej niezgodności (np. UUID jako str).

In [14]:
from pydantic import BaseModel, ValidationError
class Person(BaseModel):
    name: str
    age: int

print(Person.model_validate({'name': 'Adrian', 'age': '123'}))  # lax mode
#> x=123

try:
    Person.model_validate({'name': 'Adrian', 'age': '123'}, strict=True)  # strict mode
except ValidationError as exc:
    print(exc)

name='Adrian' age=123
1 validation error for Person
age
  Input should be a valid integer [type=int_type, input_value='123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/int_type


Tryb "strict" można włączać dla całego modelu (np.: przez `config_dict`), dla pojedynczych pól czy podczas wołania `model_validate`

## Wartości domyślne i pola
Modele Pydantica mogą mieć wartości domyślne:

In [ ]:
class Student(BaseModel):
    name: str
    gpa: decimal.Decimal = decimal.Decimal("4.5")

... jednak muszą być one stałymi, znanymi w momencie definicji pola. Wynika to z automatycznej konwersji tak zadeklarowanych pól na parametry konstruktora. Jak wiadomo, parametry domyślne funkcji są wyliczane jedynie raz, gdy moduł definiujący funkcje jest po raz pierwszy ładowany. W przypadku wartości wyliczanych dynamicznie skutkowałoby to dość zaskakującym zachowaniem. Aby jednak osiągnąć ten sam efekt, możemy posłużyć się mechanizmem tak zwanych pól (ang. *fields*):

In [23]:
import time
import uuid
from decimal import Decimal
from typing import Annotated
from pydantic import BaseModel, Field, AliasPath, AliasChoices


def fetch_gpa():
    # udajemy, że pobieramy dane z internetu:
    time.sleep(1)
    print("fetching....")
    return Decimal('4.5')

class Student(BaseModel):
    id: Annotated[str, Field(default_factory=lambda: uuid.uuid4().hex)]
    first_name: str = Field(
        validation_alias=AliasPath('name', 0),
        pattern=r'^[A-Z]\w+$'
    )
    last_name: str = Field(
        validation_alias=AliasPath('name', 1),
        pattern=r'^[A-Z]\w+$'
    )
    gpa: Decimal = Field(
        default_factory=fetch_gpa,
        validation_alias=AliasChoices('gpa', 'srednia'),
        ge=Decimal('2.0'),
        le=Decimal('5.0'),
        repr=False
    )

janek = Student(name=["Janek", "Kowalski"])
print(repr(janek))

zosia = Student(name=["Zosia", "Kowalska"], gpa=Decimal('4.8'))
print(repr(zosia))

hania = Student(name=["Hania", "Nowak"], gpa='3.3')
print(hania.gpa)

fetching....
Student(id='b025a1a4df9e46a38604d10b5f54ecfe', first_name='Janek', last_name='Kowalski')
Student(id='dd4fed7fb61f4972973284889e30b38d', first_name='Zosia', last_name='Kowalska')
3.3


*Fields* mogą służyć nie tylko do dynamicznego generowania wartości domyślnych pól, ale również definiować ich aliasy, walidować wartości numeryczne czy zgodność z wyrażeniem regularnym.

## Generowanie schemy JSON
Pydantic umożliwia proste generowanie schemy JSON (ang. *jsonschema*) z modeli lub tak naprawdę dowolnych typów:

In [20]:
import time
import uuid
import json
from decimal import Decimal
from typing import Annotated
from pydantic import BaseModel, Field

class Student(BaseModel):
    id: Annotated[str, Field(default_factory=lambda: uuid.uuid4().hex)]
    name: str
    gpa: Decimal

print(json.dumps(Student.model_json_schema(), indent=2))

{
  "properties": {
    "id": {
      "title": "Id",
      "type": "string"
    },
    "name": {
      "title": "Name",
      "type": "string"
    },
    "gpa": {
      "anyOf": [
        {
          "type": "number"
        },
        {
          "type": "string"
        }
      ],
      "title": "Gpa"
    }
  },
  "required": [
    "name",
    "gpa"
  ],
  "title": "Student",
  "type": "object"
}


In [21]:
from pydantic import TypeAdapter

adapter = TypeAdapter(list[int])
adapter.json_schema()

{'items': {'type': 'integer'}, 'type': 'array'}

## Niezmienialność (*Immutability*)
Podobnie jak `dataclasses`, modele Pydanticowe mogą być niezmienialne - służy do tego parametr `frozen=True|False`, który może działać na poziomie pól:

In [22]:
from pydantic import BaseModel, Field, ValidationError


class User(BaseModel):
    name: str = Field(frozen=True)
    age: int


user = User(name='John', age=42)

try:
    user.name = 'Jane'  
except ValidationError as e:
    print(e)

1 validation error for User
name
  Field is frozen [type=frozen_field, input_value='Jane', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/frozen_field


Tak stworzone obiekty nie mogą mieć nadpisywanych pól oznaczonych jako "frozen". Aby stworzyć ich kopię ze zmienioną wartością należy użyć `model_copy`:

In [ ]:
from pydantic import BaseModel, Field, ValidationError


class User(BaseModel):
    name: str = Field(frozen=True)
    age: int


user = User(name='John', age=42)

try:
    user2 = user.model_copy(update={'name': 'Ian'})
    print(user2)
except ValidationError as e:
    print(e)
    """
    1 validation error for User
    name
      Field is frozen [type=frozen_field, input_value='Jane', input_type=str]
    """

## Wydajność
https://docs.pydantic.dev/latest/concepts/performance/

## *Zadanie*
W aplikacjach czasem spotykamy się z sytuacją, gdzie potrzeba jest reprezentować zakres dat. Najlepiej jest wówczas posłużyć się klasą, której obiekty zenkapsulują początek i koniec zakresu, zapewniając spełnienie pewnych zdroworozsądkowych niezmienników. Taka klasa powinna walidować poprawność danych, które są używane do stworzenia jej egzemplarzy tak, by móc traktować je jako niezmienialne (*ang: immutable*). Zmodyfikuj tę klasę `DateRange`, dostępną w `examples/date_range.py` z użyciem Pydantic tak, by:
- nie możliwe było stworzenie jej obiektów gdy start jest później niż koniec (start < end)
- możliwe było stworzenie jej obiektów metodami fabrycznymi `years|months|days|hours_back`, ale tylko gdy podamy nieujemne parametry oznaczające ile lat|miesięcy|dni|godzin wstecz od obecnej daty powinien zaczynać się zakres
- metoda `from_last` akceptowała tylko napisy, w postaci, która jest parsowalna bez zmiany treści tej metody
- po utworzeniu nie było możliwe zmienianie wartości jej pól

## *Zadanie*
Alpha Vantage to serwis udostepniający dane giełdowe przez API. Jedna z odpowiedzi tego API znajduje się w pliku `examples/example_alpha_vantage_response.json`. Użyj Pydantica do jej zwalidowania, zanim będzie mogła być użyta do dalszego przetwarzania w jakiejś aplikacji.
- utwórz odpowiedni model Pydantica
- pamiętaj o poprawnym typowaniu - pola reprezentujące daty powinny być reprezentowane przez obiekty `date`, pola reprezentujące wartości akcji `Decimal` etc.
- zapewnij, że w danych nie ma logicznych błędów
    - np. że wartości `open` i `close` mieszczą się między `high` i `low`
    - ... lub że dla każdego dnia wszystkie wartości są podane (tj. `open`, `close`, `high` i `low`)
- niektóre pola mogą obecnie mieć niezbyt wygodne w użyciu nazwy, np.: `1. open`. Pola naszego modelu powinny mieć jednolicie ponazywane pola, zgodne ze standardowymi konwencjami w Pythonie, a mimo to móc parsować tego JSONa w niezmienionej formie
- wygeneruj schemę z modelu i zapisz ją w pliku `examples/alpha_vantage_schema.json`. Zadbaj by schema zawierała informację o wersji - na początek 1.0.0
- wypisz zwalidowane dane w czytelny sposób, ale tak, by symbol akcji nie został wypisany. Zamiast tego wypisz ten symbol przed wypisaniem pozostałych pól modelu